# Robot arm kinematic control

This page introduce the concept of controlling the end-effector of a robot when its joint are controlled with velocity set-points.

## Importing Librairies

In [ ]:
# Loading a robotic toolbox
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro

# Generic python tools
import numpy as np
from IPython import display

# Defining the robot arm

Here we load a already defined class from the library including the dynamic equations defining the robot arm behavior.



In [ ]:
from pyro.dynamic import manipulator

# Dynamic model (inputs are motor torques)
torque_controlled_robot    = manipulator.TwoLinkManipulator()

torque_controlled_robot.l1 = 4.0 # length of first rigid link
torque_controlled_robot.l2 = 3.0 # length of second rigid link
torque_controlled_robot.l_domain = 10

# Kinematic only model (inputs are motor velocities)
speed_controlled_robot  = manipulator.SpeedControlledManipulator.from_manipulator( torque_controlled_robot )

robot = speed_controlled_robot # For this exercise, we will only use the kinematic model

## Showing the robot behavior with constant velocity inputs

Here we run a simulation of the robot arm with no controllers:

In [ ]:
# robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
robot.x0 = np.array([ 0.1, 0.1])

# robot constant inputs
robot.ubar = np.array([ 0.5, 1.0]) # Constant joint velocities

# run the simulation
robot.compute_trajectory( tf = 6 )

# Animate and display the simulation
ani  = robot.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Kinematic controller

Here bellow you can write your own custom kinematic control law, i.e. computing the joint velocity based on actual joint position and time. Here the framework is already set in place for you to speed-up the implementation process. The following variable are pre-computed and available for you to use in the feedback law:

*   The joint angles $q$
*   The time $t$

Furthermore, based on a kinematic robot model the following properties are also pre-computed:
*   The actual end-effector position $r$
*   The Jacobian $J(q)$



In [ ]:
from pyro.control  import robotcontrollers
import matplotlib.pyplot as plt

class CustomKinematicController( robotcontrollers.EndEffectorKinematicController ) :

    def statique_of_robot(self, qmin, qmax, num_pts, Fe_vect): # qmin : int, qmax : int,
        """
          Feedback static computation u = c(y,r,t)

          INPUTS
          qmin : limit min of domaine      int
          qmax : limit max of domaine      int
          num_pts : num of points in linspace int
          Fe_vect : Effector forces vector n x 1

          OUTPUTS
          none  : plots tau (torque) surfaces 3d plot : (q1, q2, tau_i)

        """

        q1 = np.linspace(qmin, qmax, num_pts)
        q2 = np.linspace(qmin, qmax, num_pts)
        Tau1 = np.zeros((len(q1), len(q2)))
        Tau2 = np.zeros((len(q1), len(q2)))
        q1, q2 = np.meshgrid(q1, q2)

        # Pre-computed values based on the robot kinematic model
        for i in range(0, len(q1)-1):
          for j in range(0, len(q2)-1):
            q = np.array([q1[i,j], q2[i,j]])
            t = torque_controlled_robot.J( q ).T @ Fe_vect # Refaire en calculant l'inv de la matrice
            Tau1[i, j] = t[0]
            Tau2[i, j] = t[1]


        # Create a 3D plot
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the meshgrid
        ax.scatter(q1, q2, Tau1)

        # Add labels and title
        ax.set_xlabel('Q1')
        ax.set_ylabel('Q2')
        ax.set_zlabel('tau 1')
        ax.set_title('3D Meshgrid')

        # Display the plot
        plt.show()


      # Create a 3D plot
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the meshgrid
        ax.scatter(q1, q2, Tau2)

        # Add labels and title
        ax.set_xlabel('Q1')
        ax.set_ylabel('Q2')
        ax.set_zlabel('tau 2')
        ax.set_title('3D Meshgrid')

        # Display the plot
        plt.show()
    #############################
    def c( self , y , r , t ):
        """
        Feedback static computation u = c(y,r,t)

        INPUTS
        y  : sensor signal vector     p x 1
        r  : reference signal vector  k x 1
        t  : time                     1 x 1

        OUTPUTS
        dq  : joint velocity vector   m x 1

        """

        # Feedback from sensors
        q = y                  # Joint angles

        # Pre-computed values based on the robot kinematic model
        J = self.J( q )        # Jacobian computation
        r = self.fwd_kin( q )  # End-effector postion

        ##############################
        # YOUR CODE BELLOW !!
        ##############################

        # Compute the reference
        r_d  = np.zeros(2) # Place-holder
        dr_d = np.zeros(2) # Place-holder

        # Compute the desired effector velocity
        dr_r = np.array([-0.1,0.1]) # Place holder

        # From effector speed to joint speed
        dq = np.linalg.inv( J ) @ dr_r

        return dq

In [ ]:
ctl = CustomKinematicController( robot )
ctl.statique_of_robot(-np.pi, np.pi, 25, np.array([0.0, -1.0]))

## Simulating the robot in closed-loop

Here we run a simulation of the robot with the controller.

In [ ]:
# Create the closed-loop system
robot_with_controller = ctl + robot

# Run the simulation
robot_with_controller.x0[0] = 1.0
robot_with_controller.x0[1] = 1.0
robot_with_controller.compute_trajectory( tf = 5 )

In [ ]:
# Animate the simulation
ani = robot_with_controller.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

In [ ]:
# Plot systems states
robot_with_controller.plot_trajectory('x')

In [ ]:
# Plot control inputs
robot_with_controller.plot_trajectory('u')